# Step 2.5

In [32]:
import os
import json
import numpy as np

In [33]:
# Load dataset IDs
def load_json(filepath):
    with open(filepath, "r") as f:
        return json.load(f)

train_ids = load_json("Splits/train_ids.json")
val_ids = load_json("Splits/val_ids.json")
test_ids = load_json("Splits/test_ids.json")

# Load correctness labels
with open("Labels/labels_lumbar_error.json") as f:
    labels = json.load(f)

In [34]:
# Function to load pose annotations from YOLO-Pose `.txt` files
def load_pose_annotations(image_ids, annotation_folder):
    pose_data = []

    for img_id in image_ids:
        annotation_file = os.path.join(annotation_folder, f"{img_id}.txt")
        
        if os.path.exists(annotation_file):
            with open(annotation_file, "r") as f:
                lines = f.readlines()

            # Each .txt file may contain multiple people, so loop over lines
            for line in lines:
                data = list(map(float, line.strip().split()))
                keypoints = np.array(data[1:]).reshape(-1, 3)  # Reshape to (num_keypoints, 3)
                pose_data.append({"id": img_id, "keypoints": keypoints})
    
    return pose_data

In [16]:
# def load_pose_annotations(image_ids, annotation_folder, expected_keypoints=17):
#     pose_data = []

#     for img_id in image_ids:
#         annotation_file = os.path.join(annotation_folder, f"{img_id}.txt")

#         if os.path.exists(annotation_file):
#             with open(annotation_file, "r") as f:
#                 lines = f.readlines()

#             # Each .txt file may contain multiple people, so loop over lines
#             for line in lines:
#                 data = list(map(float, line.strip().split()))

#                 # Ensure there are enough keypoints (expected: class_id + keypoints * 3)
#                 if len(data) != (expected_keypoints * 3 + 1):
#                     print(f"Warning: {annotation_file} has incorrect keypoint format. Skipping.")
#                     continue

#                 keypoints = np.array(data[1:]).reshape(-1, 3)  # Reshape to (num_keypoints, 3)
#                 pose_data.append({"id": img_id, "keypoints": keypoints})
    
#     return pose_data

In [17]:
# def load_pose_annotations(image_ids, annotation_folder, expected_keypoints=17):
#     pose_data = []

#     for img_id in image_ids:
#         annotation_file = os.path.join(annotation_folder, f"{img_id}.txt")

#         if os.path.exists(annotation_file):
#             with open(annotation_file, "r") as f:
#                 lines = f.readlines()

#             for line in lines:
#                 data = list(map(float, line.strip().split()))

#                 # Ensure correct keypoint format (class_id + keypoints * 3)
#                 if len(data) != (expected_keypoints * 3 + 1):
#                     print(f"Skipping {annotation_file}: Incorrect format ({len(data)} values).")
#                     continue

#                 # Extract keypoints and remove class ID
#                 keypoints = np.array(data[1:]).reshape(-1, 3)  # Shape (num_keypoints, 3)

#                 # Remove keypoints where confidence == 0
#                 keypoints = keypoints[keypoints[:, 2] > 0]

#                 # If no valid keypoints remain, skip the annotation
#                 if keypoints.shape[0] == 0:
#                     print(f"Skipping {annotation_file}: No valid keypoints detected.")
#                     continue

#                 pose_data.append({"id": img_id, "keypoints": keypoints})
    
#     return pose_data

In [36]:
annotation_path_train = "runs/pose/predict/train/labels/"
annotation_path_val = "runs/pose/predict/val/labels/"
# annotation_path = "runs/pose/predict/test/labels/"

# Load training, validation, and test pose data
train_pose = load_pose_annotations(train_ids, annotation_path_train)
val_pose = load_pose_annotations(val_ids, annotation_path_val)
# test_pose = load_pose_annotations(test_ids, annotation_path)

print(f"Loaded {len(train_pose)} training samples")
print(f"Loaded {len(val_pose)} validation samples")
# print(f"Loaded {len(test_pose)} test samples")



Loaded 14342 training samples
Loaded 2913 validation samples


In [ ]:
# print(test_pose)#[0]["keypoints"].shape)


[{'id': '54105_1_11', 'keypoints': array([[6.04218e-01, 3.84414e-01, 4.23910e-01],
       [7.68041e-01, 4.23679e-01, 3.99679e-02],
       [8.67327e-01, 4.29498e-01, 1.55747e-02],
       [2.12949e-01, 4.87956e-01, 1.13225e-04],
       [5.83819e-02, 5.80669e-01, 1.01366e-01],
       [9.84251e-01, 5.79687e-01, 1.05404e-01],
       [7.83542e-01, 5.95793e-01, 2.94949e-01],
       [2.07646e-01, 4.77259e-01, 3.09969e-01],
       [2.08317e-01, 7.06330e-01, 4.20065e-01],
       [9.76155e-01, 7.02144e-01, 4.18215e-01],
       [9.44174e-01, 6.66101e-01, 6.96014e-01],
       [8.91556e-01, 6.74202e-01, 6.94176e-01]])}, {'id': '55787_3_57', 'keypoints': array([[0.493462, 0.652383, 0.462609],
       [0.690471, 0.274767, 0.388075],
       [0.917141, 0.282506, 0.376247],
       [0.247627, 0.330493, 0.377449],
       [0.036846, 0.404514, 0.440233],
       [0.995879, 0.413402, 0.437244],
       [0.842601, 0.447536, 0.566487],
       [0.44602 , 0.44341 , 0.695863],
       [0.978975, 0.456077, 0.675453],
 

In [23]:
# %pip install torch_geometric


  Using cached attrs-25.1.0-py3-none-any.whl.metadata (10 kB)
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   --------- ------------------------------ 0.3/1.1 MB ? eta -:--:--
   ------------------ --------------------- 0.5/1.1 MB 2.1 MB/s eta 0:00:01
   ---------------------------------------- 1.1/1.1 MB 1.7 MB/s eta 0:00:00
Using cached attrs-25.1.0-py3-none-any.whl (63 kB)


# Step 3: Convert Pose Data to Graph

In [37]:
import torch
import networkx as nx
from torch_geometric.data import Data

In [38]:
# Define COCO skeleton keypoint connections (edges)
SKELETON_EDGES = [
    (0, 1), (1, 2), (2, 3), (3, 4),  # Head
    (0, 5), (5, 6), (6, 7), (7, 9),  # Left arm
    (0, 6), (6, 8), (8, 10),  # Right arm
    (5, 11), (6, 12),  # Upper body
    (11, 12),  # Pelvis
    (11, 13), (13, 15),  # Left leg
    (12, 14), (14, 16)   # Right leg
]

In [39]:
# Function to convert pose data into graph format for GCN
def create_pose_graph(pose_data):
    graphs = []
    
    for item in pose_data:
        keypoints = item["keypoints"]
        img_id = item["id"]

        # Create nodes (keypoints as graph nodes)
        node_features = torch.tensor(keypoints[:, :2], dtype=torch.float)  # Use (x, y) coordinates
        
        # Define edges (using skeleton structure)
        edge_index = torch.tensor(SKELETON_EDGES, dtype=torch.long).t().contiguous()

        # Get correctness label from JSON
        label = torch.tensor([labels.get(img_id, 0)], dtype=torch.long)  # Default to incorrect (0)

        # Create PyTorch Geometric Data object
        graph = Data(x=node_features, edge_index=edge_index, y=label)
        graphs.append(graph)
    
    return graphs

In [40]:
# Convert pose data into graph format
train_graphs = create_pose_graph(train_pose)
val_graphs = create_pose_graph(val_pose)
# test_graphs = create_pose_graph(test_pose)

In [41]:
# print(test_graphs[0])

In [42]:
# Save graphs for training
torch.save(train_graphs, "train_pose_graphs.pt")
torch.save(val_graphs, "val_pose_graphs.pt")
# torch.save(test_graphs, "test_pose_graphs.pt")